In [1]:
#move to the folder where the scraped data is on your device
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Data/WiChat/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Data/WiChat


In [2]:
!pip install virtualenv
!virtualenv /content/drive/MyDrive/Data/WiChat
!source /content/drive/MyDrive/Data/WiChat/bin/activate

created virtual environment CPython3.11.11.final.0-64 in 14516ms
  creator CPython3Posix(dest=/content/drive/MyDrive/Data/WiChat, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.0.1, setuptools==75.8.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
# install dependencies
!pip install -qU langchain
!pip install -qU transformers
!pip install -qU langchain-community
!pip install -qU unstructured
!pip install -qU sentence-transformers
!pip install -qU faiss-cpu
!pip install -qU langchain_openai
!pip install -qU openai
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━

In [3]:
import openai
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser


In [4]:
#API_KEY = os.environ["OPENAI_API_KEY"]
#openai.apikey = APIKEY
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
import os
os.environ['OPENAI_API_KEY'] = key
# ensure the notebook is in the same folder as the data files
# load all txt files
def load_data(path):
  loader1 = DirectoryLoader(path, glob = '*.txt', show_progress = True)
  # get content of txt files
  docs = loader1.load()

  return docs


def get_chunks(docs):

    # split the txt files into chunks of 1000 characters and 150 characters overlap
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 150)
    chunks = text_splitter.split_documents(docs)


    return chunks

path = '/content/drive/MyDrive/Data/WiChat/scraped_data'
docs = load_data(path)

data = get_chunks(docs)

100%|██████████| 90/90 [00:22<00:00,  3.95it/s]


In [5]:


# embed data sources
def embed(data, device, model):
  model_kwargs = {'device': device}
  encode_kwargs = {'normalize_embeddings': False}

  embeddings = HuggingFaceEmbeddings(
    model_name = model,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
  )
  return embeddings

def store_data(data, embeddings):
  # vector store
  db = FAISS.from_documents(data, embeddings)
  return db

embeddings = embed(data, 'cpu', 'sentence-transformers/all-MiniLM-L6-v2')
db = store_data(data, embeddings)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
llm = ChatOpenAI(model = "gpt-4o")

# Define the chat prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are called WiChat, which is short for Worldbank Ideas Chatbot, the chatbot for the Worldbank Ideas Project. You are friendly and follow instructions to answer questions extremely well. Please be truthful and give direct answers. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the response short and concise in at most five sentences. If the user chats in a different language, translate accurately and respond in the same language. You will provide specific details and accurate answers to user queries on the Worldbank Ideas Project."),
         MessagesPlaceholder("chat_history"),
        ("human", "Use only the retrieved {context} to answer the user question {input}.")
    ]
)

# --- Create RAG chain ---

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
### Statefully manage chat history ###

messages_history = {}

def get_session_history(session_id: str):
    if session_id not in messages_history:
        messages_history[session_id] = ChatMessageHistory()
    return messages_history[session_id]



In [13]:
retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


# --- Response Generation ---
def generate_response(query):

    return conversational_rag_chain.invoke({"input": query}, config={"configurable": {"session_id": "1"}})["answer"]

query = "What is worldbank ideas project"
response = generate_response(query)


response

'The World Bank IDEAS Project involves a collaboration between the World Bank and local entities like the Ministry of Education to review the ongoing project implementations. It focuses on conducting detailed assessments and strategic discussions to enhance project outcomes and ensure they align with developmental objectives. This includes visits, like those in Edo State, Nigeria, for comprehensive reviews and to ensure sustainable development. The project supports the implementation of educational and developmental reforms, monitored by international and local stakeholders.'

In [14]:

query = "Why are you called WiChat?"
response = generate_response(query)


response

"I am called WiChat, short for Worldbank Ideas Chatbot, as I am designed to help answer queries related to the Worldbank Ideas Project. My name reflects my role in facilitating discussions and providing information about the initiatives, projects, and assessments carried out by the World Bank's Ideas Project. I aim to provide seamless communication and accurate information to users asking about the Worldbank's strategic efforts and developments."

In [15]:
query = "World Bank Ideas project, c'est quoi?"
response = generate_response(query)


response

"Le projet Worldbank IDEAS implique une collaboration entre la Banque mondiale et des entités locales telles que le Ministère de l'Éducation pour examiner la mise en œuvre continue des projets. Il s'agit d'effectuer des évaluations détaillées et des discussions stratégiques pour optimiser les résultats et garantir qu'ils s'alignent sur les objectifs de développement. Ces initiatives incluent des visites, comme celles dans l'État d'Edo, au sud du Nigeria, pour des revues complètes et pour assurer un développement durable. Le projet soutient la mise en œuvre de réformes éducatives et de développement, surveillées par des parties prenantes internationales et locales."

In [17]:
query = "What role does Uniccon play in the World Bank IDEAS training?"
response = generate_response(query)


response

"Uniccon Group contributes to the World Bank IDEAS project by leveraging its technological capabilities to enhance the project's implementation and outcomes. As part of the broad initiative involving various stakeholders like the Ministry of Education, Uniccon Group plays a role in innovation and technology-driven components. They are involved in activities that align with the project's objective to address deficiencies in the education system, particularly around technical training and project monitoring. Their involvement assists in strategic discussions and assessments, supporting sustainable development goals."

In [19]:
query = "How can I join the World Bank IDEAS digital literacy training. What coures can I learn and which organisations will handle the training?"
response = generate_response(query)


response

'To join the World Bank IDEAS digital literacy training, you can look into the Innovation Grant Facility (IGF) under the IDEAS Project, which supports innovative training in digital skills specifically in the North-West catchment area. The training is facilitated by private organizations awarded grants through IGF to deliver programs in digital literacy. It is part of a broader effort involving the Federal Ministry of Education and the National Board for Technical Education with implementing agencies in states such as Kano, Abia, Edo, Ekiti, Benue, and Gombe. The courses you might learn include digital skills that align with local demands and innovative industry needs. For specific details on enrolling, it is best to contact the Kano IDEAS State Implementation Unit or related agencies in your region.'